# Before your start:
- Read the README.md file
- Comment as much as you can and use the resources (README.md file)
- Happy learning!

In [74]:
# First I load my libraries 

import numpy as np 
import pandas as pd


# Challenge 1 - Index Functions

In this challenge, we will practice the advanced functions introduced in the lesson. However, before we start, we will load and evaluate our dataset. 

Load the dataset from Ironhack's database:
* db: `admissions predict`
* table: `admissions predict`

In [75]:
# your code here
db = pd.read_csv('/Users/jossuebangos/Documents/GitHub/lab-advanced-pandas/your-code/data/Admission_Predict.csv')
db.head()


,Serial No.,GRE Score,TOEFL Score,University Rating,SOP,LOR,CGPA,Research,Chance of Admit
0,1,337,118,4,4.5,4.5,9.65,1,0.92
1,2,316,104,3,3.0,3.5,8.00,1,0.72
2,3,322,110,3,3.5,2.5,8.67,1,0.80
3,4,314,103,2,2.0,3.0,8.21,0,0.65
4,5,330,115,5,4.5,3.0,9.34,1,0.90


Remove trailing spaces at the end of the column names if there are any.


In [76]:
#clean_db[''] = db[''].strip() 

# iterating the columns , print the columns to check if I should Strip () something. 

db.columns = db.columns.str.strip()
# for col in db.columns: 
#     print(col) 

Let's evaluate the dataset by looking at the `head` function.

In [77]:
db.head()

,Serial No.,GRE Score,TOEFL Score,University Rating,SOP,LOR,CGPA,Research,Chance of Admit
0,1,337,118,4,4.5,4.5,9.65,1,0.92
1,2,316,104,3,3.0,3.5,8.00,1,0.72
2,3,322,110,3,3.5,2.5,8.67,1,0.80
3,4,314,103,2,2.0,3.0,8.21,0,0.65
4,5,330,115,5,4.5,3.0,9.34,1,0.90


Before beginning to work with this dataset and evaluating graduate admissions data, we will verify that there is no missing data in the dataset. Do this in the cell below.

In [78]:
# https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.isnull.html
#This will drop entire rows or cells? If only cells is fine, else I need to do soemthing else such as replace. 


# db2= db.dropna() 
# db2.head()

db.isna().sum()


Serial No.           0
GRE Score            0
TOEFL Score          0
University Rating    0
SOP                  0
LOR                  0
CGPA                 0
Research             0
Chance of Admit      0
dtype: int64

Interestingly, there is a column that uniquely identifies the applicants. This column is the serial number column. Instead of having our own index, we should make this column our index. Do this in the cell below. Keep the column in the dataframe in addition to making it an index.

In [79]:
#https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.set_index.html

#Set index 
#db.set_index('Serial No.',inplace=True)
db['Serial No index'] = db.set_index('Serial No.', inplace=False).index #Make a duplicate of the column that needs to be the index
db_index = db.set_index('Serial No index') #Set the new column to be the index, in a new df
db_index

,Serial No.,GRE Score,TOEFL Score,University Rating,SOP,LOR,CGPA,Research,Chance of Admit
Serial No index,,,,,,,,,
1,1,337,118,4,4.5,4.5,9.65,1,0.92
2,2,316,104,3,3.0,3.5,8.00,1,0.72
3,3,322,110,3,3.5,2.5,8.67,1,0.80
4,4,314,103,2,2.0,3.0,8.21,0,0.65
5,5,330,115,5,4.5,3.0,9.34,1,0.90
...,...,...,...,...,...,...,...,...,...
381,381,324,110,3,3.5,3.5,9.04,1,0.82
382,382,325,107,3,3.0,3.5,9.11,1,0.84
383,383,330,116,4,5.0,4.5,9.45,1,0.91


In [80]:
#First prove that they uniquely identify the data 

print(db.set_index(['GRE Score','CGPA']).index.is_unique)


True



Replace the index with an index comprised of two columns - `GRE Score` and `CGPA`. Remove the columns from the dataset as well. Make sure to use `inplace=True`.

Now change the index back to a sequence starting at zero using the `reset_index` function. Make sure to use `inplace=True`.

In [81]:
#https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.reset_index.html

#db.set_index(['GRE Score','CGPA'],inplace = True)
#print(db)
db.reset_index(inplace=True)
db



,index,Serial No.,GRE Score,TOEFL Score,University Rating,SOP,LOR,CGPA,Research,Chance of Admit,Serial No index
0,0,1,337,118,4,4.5,4.5,9.65,1,0.92,1
1,1,2,316,104,3,3.0,3.5,8.00,1,0.72,2
2,2,3,322,110,3,3.5,2.5,8.67,1,0.80,3
3,3,4,314,103,2,2.0,3.0,8.21,0,0.65,4
4,4,5,330,115,5,4.5,3.0,9.34,1,0.90,5
...,...,...,...,...,...,...,...,...,...,...,...
380,380,381,324,110,3,3.5,3.5,9.04,1,0.82,381
381,381,382,325,107,3,3.0,3.5,9.11,1,0.84,382
382,382,383,330,116,4,5.0,4.5,9.45,1,0.91,383
383,383,384,312,103,3,3.5,4.0,8.78,0,0.67,384


# Challenge 2 - Advanced Functions

In this part of the lab, we would like to test complex conditions on the entire dataset at once. Let's start by finding the number of rows where the CGPA is higher than 9 and the student has conducted research.

In [82]:

# total_spent.loc[total_spent['amount_spent'] > total_spent['amount_spent'].quantile(0.75), 'Costumer Label'] = 'PC'

#db3 = db.loc[db['CGPA']] > 9 & db.loc[['Research']] == 1

db4 = db[(db['CGPA'] > 9) & (db['Research'] == 1)].count()
db4

index                101
Serial No.           101
GRE Score            101
TOEFL Score          101
University Rating    101
SOP                  101
LOR                  101
CGPA                 101
Research             101
Chance of Admit      101
Serial No index      101
dtype: int64

Now return all the rows where the CGPA is greater than 9 and the SOP score is less than 3.5. Find the mean chance of admit for these applicants.

In [83]:
# your code here
chance = db[(db['CGPA']>9)&(db['SOP']<3.5)]
chance['Chance of Admit'].mean()
#db3['NewCol']= db3['NewCol'].mean()


0.8019999999999999

We would like to create a deciding factor column for each student. We standardize several columns and then pick the most important factor from a lookup table. If the standardized value is above 0.8, the student will be accepted.

We will start by creating a standardized column for `CGPA`, `GRE Score` and `LOR`. We will name these columns `CGPA_std`, `GRE_std`, and `LOR_std` respecively.

Recall that standardizing a column is done by subtracting the mean of the column from all observations in the column and then dividing each observation in the column by the column's standard deviation.

In the cell below, write a standardization function (a function that takes a column as input and returns a standardized column as output). Make sure to use the `numpy` versions of mean and standard deviation.

In [84]:
# What us numpy version of mean and std compared to pandas? - n-1 , numpy has no variable filled -
#https://stackoverflow.com/questions/24984178/different-std-in-pandas-vs-numpy

#Steps
#We will start by creating a standardized column for CGPA, 
#GRE Score and LOR. We will name these columns CGPA_std, GRE_std, and LOR_std respecively

#Instructions :     # your code here, each row df.[] .=mean from all values- substract the mean and divide by std. dev. = x 


def standardize(col):
    # np.mean(col) np.std(col)
    return (col-np.mean(col))/np.std(col) 
    
#lIBRARY, there is one. 
    
    

Now create the standardized columns `CGPA_std`, `GRE_std`, and `LOR_std` and add them to the `admissions` dataframe.

In [85]:
admissions = pd.DataFrame()

admissions['CGPA_std']= standardize(db['CGPA'])
admissions['GRE_std']= standardize(db['GRE Score'])
admissions['LOR_std']= standardize(db['LOR'])

admissions

,CGPA_std,GRE_std,LOR_std
0,1.750174,1.755663,1.193197
1,-0.992501,-0.063450,0.076840
2,0.121191,0.456297,-1.039517
3,-0.643433,-0.236698,-0.481338
4,1.234884,1.149292,-0.481338
...,...,...,...
380,0.736216,0.629546,0.076840
381,0.852571,0.716170,0.076840
382,1.417729,1.149292,1.193197
383,0.304036,-0.409947,0.635019


We will generate the decision choice at random using the code below. Please run the cell.

In [86]:
# Libraries
from random import choices

In [87]:
std_columns = ['CGPA_std', 'GRE_std', 'LOR_std']

decision_choice = choices(std_columns, k=admissions.shape[0])
#decision_choice

Now create the deciding column using the `lookup` function. The lookup column is `decision_choice` found above. Call the column resulting from the lookup function `deciding_column` and add it to the `admissions` dataframe.

In [88]:
#df['value'] = df.lookup(df.index, df['best'])
admissions['deciding_column'] =  admissions.lookup(admissions.index,decision_choice)
admissions



,CGPA_std,GRE_std,LOR_std,deciding_column
0,1.750174,1.755663,1.193197,1.193197
1,-0.992501,-0.063450,0.076840,0.076840
2,0.121191,0.456297,-1.039517,0.121191
3,-0.643433,-0.236698,-0.481338,-0.236698
4,1.234884,1.149292,-0.481338,1.234884
...,...,...,...,...
380,0.736216,0.629546,0.076840,0.629546
381,0.852571,0.716170,0.076840,0.076840
382,1.417729,1.149292,1.193197,1.193197
383,0.304036,-0.409947,0.635019,0.304036


Create a column called `decision` in the `admissions` dataframe. Assign 1 to this column if the value of `deciding_column` is greater than 0.8 and 0 otherwise.

In [89]:
#np.where(a < 5, a, 10*a)
#https://numpy.org/doc/stable/reference/generated/numpy.where.html

#admissions['decision'] = admissions['deciding_column'].apply(lambda x: 1 if x > 0.8 else 0) 

admissions['Decision'] = np.where(admissions['deciding_column'] > 0.8, 1,0)


admissions.head()


,CGPA_std,GRE_std,LOR_std,deciding_column,Decision
0,1.750174,1.755663,1.193197,1.193197,1
1,-0.992501,-0.063450,0.076840,0.076840,0
2,0.121191,0.456297,-1.039517,0.121191,0
3,-0.643433,-0.236698,-0.481338,-0.236698,0
4,1.234884,1.149292,-0.481338,1.234884,1


How many applicants will be accepted to the program using the decision column? Compute the result below.

In [90]:
admissions['Decision'].sum()

87

# Challenge 3 - Method Chaining

To increase our coding efficiency, let's make a number of changes to our dataframe in one line of code.

In the cell below, remove all non character symbols from the column names, replace all spaces in column names to underscores, and change all upper case characters to lower case. Assign these new column values to `admissions.columns`.

In [92]:
# your code here
admissions.columns=admissions.columns.str.replace('[^a-zA-Z]','').str.replace(' ','_').str.lower()
admissions.columns



Index(['cgpastd', 'grestd', 'lorstd', 'decidingcolumn', 'decision'], dtype='object')

In the cell below, give all student with a university rating of 4 or higher a 10 point boost on their GRE score and split the column into 4 bins using the `cut` function. Assign this new score to the variable `adjusted_gre`.

In [99]:
# your code here
#https://www.geeksforgeeks.org/pandas-cut-method-in-python/
db.head()

db['GRE Score'] = np.where(db['University Rating'] >= 4,db['GRE Score']+10,db['GRE Score'])
db['adjusted_gre'] = pd.cut(db['GRE Score'],4)
db

,index,Serial No.,GRE Score,TOEFL Score,University Rating,SOP,LOR,CGPA,Research,Chance of Admit,Serial No index,adjusted_gre
0,0,1,377,118,4,4.5,4.5,9.65,1,0.92,1,"(357.5, 380.0]"
1,1,2,316,104,3,3.0,3.5,8.00,1,0.72,2,"(312.5, 335.0]"
2,2,3,322,110,3,3.5,2.5,8.67,1,0.80,3,"(312.5, 335.0]"
3,3,4,314,103,2,2.0,3.0,8.21,0,0.65,4,"(312.5, 335.0]"
4,4,5,370,115,5,4.5,3.0,9.34,1,0.90,5,"(357.5, 380.0]"
...,...,...,...,...,...,...,...,...,...,...,...,...
380,380,381,324,110,3,3.5,3.5,9.04,1,0.82,381,"(312.5, 335.0]"
381,381,382,325,107,3,3.0,3.5,9.11,1,0.84,382,"(312.5, 335.0]"
382,382,383,370,116,4,5.0,4.5,9.45,1,0.91,383,"(357.5, 380.0]"
383,383,384,312,103,3,3.5,4.0,8.78,0,0.67,384,"(289.91, 312.5]"
